# Produce a Convolutional Neural Network that can distinuish between different Seal Vocalisations

We have produced 480 npz files of spectrogram data for different seal data. 

In [16]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [27]:
# function to load the npz files

folder_path = 'data/processed/NPZ_files'
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.npz')]

data = []
labels = []

for file in files:
    npz_data = np.load(file)
    spectrogram = npz_data['spectrogram']
    label = npz_data['annotation']
    
    data.append(spectrogram)
    labels.append(label)

data = np.array(data)
labels = np.array(labels)

data.shape


array(['Rupe A', 'Rupe A', 'Rupe B', 'Rupe B', 'Rupe C', 'Rupe B',
       'Rupe B', 'Rupe B', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe A',
       'Rupe B', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe B', 'Rupe A',
       'Rupe B', 'Rupe B', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe A',
       'Rupe A', 'Rupe A', 'Rupe C', 'Rupe A', 'Rupe A', 'Rupe B',
       'Rupe A', 'Rupe B', 'Rupe B', 'Rupe A', 'Rupe A', 'Rupe B',
       'Rupe B', 'Rupe B', 'Rupe B', 'Rupe A', 'Rupe A', 'Rupe A',
       'Rupe A', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe B',
       'Rupe B', 'Rupe A', 'Rupe B', 'Rupe A', 'Rupe A', 'Rupe A',
       'Rupe A', 'Rupe A', 'Rupe A', 'Rupe B', 'Rupe A', 'Rupe A',
       'Rupe B', 'Rupe A', 'Rupe A', 'Rupe B', 'Rupe B', 'Rupe B',
       'Rupe B', 'Rupe B', 'Rupe C', 'Rupe A', 'Rupe A', 'Rupe A',
       'Rupe A', 'Rupe A', 'Rupe A', 'Rupe B', 'Rupe C', 'Rupe A',
       'Rupe A', 'Rupe A', 'Rupe A', 'Rupe A', 'Rupe B', 'Rupe B',
       'Rupe B', 'Rupe B', 'Rupe B', 'Rupe B', 'Rupe B', 'Rupe

Our data is only 3D, it doesnt have a channel dimension. CNN's require a 4D [shape](https://stackoverflow.com/questions/60157742/convolutional-neural-network-cnn-input-shape). We need to add a channel dimension to our data. As the spectrograms are grayscale, we will add a channel dimension of 1.

In [28]:
# Add a channel dimension to the data
data = np.expand_dims(data, axis=-1)  # Shape becomes (480, 1025, 561, 1)

In [30]:
# Normalize data
data = data / np.max(data)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)


X_train, X_test, y_train, y_test = train_test_split(data, categorical_labels, test_size=0.2, random_state=42)


In [31]:
# Assuming data shape: (samples, height, width, channels)
input_shape = X_train.shape[1:]

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(categorical_labels.shape[1], activation='softmax')  # Output layer
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\kolea\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=32,
                    verbose=1)

https://stackoverflow.com/questions/60157742/convolutional-neural-network-cnn-input-shape